In [ ]:
import pandas as pd
import numpy as np
import copy
from sklearn.model_selection import train_test_split
import math
!pip install anytree

from anytree import Node, RenderTree

     |████████████████████████████████| 51kB 3.7MB/s 


In [ ]:
path='/content/drive/MyDrive/ml-hw-1/data.xls'
def main(path):
  X_train, y_train,X_test,y_test=read_data(path)
  print('**********************************making the tree:**********************************')
  tree = Tree()
  print("**********************************train step:**********************************")
  tree.train(X_train, y_train)
  print('**********************************tree representation:**********************************')
  tree.print()
  print('test step:')
  print(tree.cal_acc(X_test,y_test))
  print('**********************************Error Prone Step:**********************************')
  error_prone(tree,X_train, y_train,X_test,y_test)
  print('**********************************5fold cross val result:**********************************')
  run_5fold_CV(X_train,y_train,X_test,y_test)

main(path)

**********************************making the tree:**********************************
**********************************train step:**********************************
data #1
data #51
data #101
data #151
data #201
data #251
**********************************tree representation:**********************************
Medu
├── 1
│   └── Fedu
│       ├── 2
│       │   └── +
│       └── 1
│           └── reason
│               ├── course
│               │   └── -
│               └── home
│                   └── Fjob
│                       └── other
│                           └── Mjob
│                               ├── at_home
│                               │   └── -
│                               └── other
│                                   └── +
├── 2
│   └── -
├── 3
│   └── Fedu
│       ├── 2
│       │   └── reason
│       │       ├── course
│       │       │   └── -
│       │       └── reputation
│       │           └── +
│       ├── 4
│       │   └── reason
│       │       └── course
│ 

In [ ]:
def read_data(path):
  N=400
  dataset=pd.read_excel(path)
  dataset['Fedu'] = dataset['Fedu'].astype(str)
  dataset['Medu'] = dataset['Medu'].astype(str)
  dataset = dataset.drop_duplicates(subset=dataset.drop(['Label'], axis=1).columns, keep='first')
  dataset.head()
  X=dataset.drop(['Label'],axis=1)
  y=dataset['Label']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
  X_train=X_train.iloc[:N,:].reset_index(drop=True)
  y_train=y_train.iloc[:N].reset_index(drop=True)
  X_test=X_test.iloc[:N,:].reset_index(drop=True)
  y_test=y_test.iloc[:N].reset_index(drop=True)
  return X_train,y_train,X_test,y_test

In [ ]:
pd.concat([X_train,y_train],axis=1).tail()

,school,sex,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,Label
95,MS,F,U,GT3,T,2,3,at_home,services,course,father,yes
96,GP,M,R,GT3,T,4,3,services,other,reputation,mother,yes
97,GP,F,U,GT3,T,1,1,other,services,course,father,no
98,GP,F,U,LE3,T,4,3,other,other,home,other,no
99,GP,F,U,GT3,T,2,1,other,other,course,mother,no


In [ ]:
class Tree:

  def __init__(self):
    self.root = My_Node()

  def train(self, X_train, y_train):
    for index,row in X_train.iterrows():
      lab=pd.DataFrame({'Label':[y_train.iloc[index]]})
      ro=row.to_frame().T
      self.root.train(ro,lab)
      if ( index%50==0):
        print('data #'+str(index+1))
      

  def print(self):
    self.root.print()

  def cal_acc(self,x,y):
    # print('in here')
    return self.root.cal_acc(x,y)  

  def reducing_error_prone(self,depth):
    self.root.reducing_error_prone(depth)

  def print_shape(self):
    return self.root.print_shape()     


In [ ]:


class My_Node(object):
    def __init__(self):
        self.attslist=[]
        self.children={}
        self.value = None
        self.label=None
        self.X=None
        self.y=None
        self.depth = 0
        self.parent = None

    def train(self,X,y):
      if (len(self.attslist)==0):
        self.attslist=X.keys().tolist()
        self.X=X
        self.y=y
      else:
        self.X=self.X.append(X,ignore_index=True)
        self.y=self.y.append(y,ignore_index=True)   
      self.pullup_or_expand(X,y)  
      

    def cal_ent_for_all(dic): #calculate entropy
      if (len(dic)==0):
        return None
      newdic={}
      for key in dic:
        total_weight=0
        total_ent=0
        for subkey in dic[key]:
          posneg=dic[key][subkey]
          pos=posneg[0]
          neg=posneg[1]
          weight=pos+neg
          total_weight+=weight
          total_ent+=weight*My_Node.entropy(posneg)
        newdic[key]=total_ent/total_weight  
      if len(newdic)==1:
        return list(newdic)[0]
      else:  
        bestatt=min(newdic, key=newdic.get)
        bestent=newdic[bestatt]
        return bestatt,bestent,newdic

    def make_dic(lst,posneglst):
      dic={}
      for i in range(len(lst)):
        dic[lst[i]]=posneglst[i]
      return dic  


    def yes_no_count(df):
      yes,no=0,0
      for index,row in df.iterrows():
        if row['Label']=='yes':
          yes+=1
        elif row['Label']=='no':
          no+=1  
      return np.array([yes,no])

    def split_to_posnegs(self):
      list_of_atts=self.attslist
      df=self.X.copy()
      df=pd.concat([df,self.y],axis=1)
      dic={}
      total_posneg=My_Node.yes_no_count(df)
      for att in list_of_atts:
        grouped_df=df.groupby([att]).apply(lambda x: x)
        lst=grouped_df[att].unique()
        uniques=([x for x in lst])
        att_ent=0
        lst_att_values=[]
        lst_att_values_posnegs=[]
        for unique in uniques:
          new_df=grouped_df[[att,'Label']]
          new_df=new_df[grouped_df[att]==unique]
          posneg=My_Node.yes_no_count(new_df)
          lst_att_values.append(unique)
          lst_att_values_posnegs.append(posneg)
        dic[att]=My_Node.make_dic(lst_att_values,lst_att_values_posnegs)
      return dic  

    def entropy(posneg):
      pos=posneg[0]
      neg=posneg[1]
      t1=(pos/(pos+neg))
      t2=(neg/(pos+neg))
      if t1==0:
        return -t2*math.log2(t2)
      elif t2==0:
        return -t1*math.log2(t1)
      else:    
        return -( t1*math.log2(t1)+ t2*math.log2(t2))

    # def cal_ent_for_all():
    #   df=df=pd.concat([self.X,self.y],axis=1)
    #   for att in self.attslist:
    #     gp=df[].groupby([feature_name])
          # grouped_data = data.groupby([attr]) # attr is a string
          # grouped_data_count = grouped_data.count()
          # group_names = grouped_data.apply(lambda x: x)[attr].unique()
          # result = 0
        

    def choose_best_att(self):
      # print('nabayad biad')
      # print('choosing best att')
      if (self.y[self.y['Label']=='yes'].shape[0]==self.y.shape[0] ):
        self.label=1
        return None
      elif (self.y[self.y['Label']=='no'].shape[0]==self.y.shape[0]):
        self.label=0
        return None
      dic=self.split_to_posnegs()
      # print(dic)
      try:
        bestatt,bestent,newdic=My_Node.cal_ent_for_all(dic)
        # print(newdic)
      except:
        term=My_Node.cal_ent_for_all(dic)
        # print(term)
        return term
      if self.value != None :
        if (bestent==newdic[self.value]):
          best=self.value
          return best
      best=bestatt
      return best


    



    def pullup_or_expand(self,X_train=None,y_train=None):
      # print('in pull up or expand')
      tempatt=self.choose_best_att()
      if tempatt==None:
        return  
      if self.value==None:
        self.value=tempatt
        self.expand()
      elif self.value != tempatt:
        self.pull_up(tempatt)
        self.update_tree_with_a_data(X_train, y_train)
      else:
        self.update_tree_with_a_data(X_train,y_train)  



    def expand(self,attribute=None):
      # print('in expand')
      # print(self.value)
      if attribute == None:
        feature_name = self.value
      else:
        feature_name = attribute
        self.value=feature_name
      df=pd.concat([self.X, self.y],axis=1)
      feature_name=self.value
      self.label=None
      # print(self.value)
      # print('***********')
      for name, group in df.groupby([feature_name]):
        child=My_Node()
        child.X=group.drop(['Label'], axis=1)
        child.y=group['Label'].to_frame()
        child.depth=self.depth + 1
        child.attslist=self.attslist.copy()
        child.attslist.remove(feature_name)
        child.parent=self
        self.children[name]=child
        child.pullup_or_expand()
      return

    def update_tree_with_a_data(self,X_train,y_train):
      # print('in update trree')
      if (X_train[self.value].iloc[0] in self.children.keys()):
        child=self.children[X_train[self.value].iloc[0] ]    
        # print('shape')
        # print(child.X.shape)
        # print(child.y.shape)

        child.X=child.X.append(X_train,ignore_index=True)
        child.y=child.y.append(y_train,ignore_index=True)
      else:  
        child=My_Node()
        child.X=X_train
        y_train.index=X_train.index
        child.y=y_train
        child.depth=self.depth + 1
        child.attslist=self.attslist.copy()
        child.attslist.remove(self.value)
        child.parent=self
        child.pullup_or_expand()
        self.children[X_train[self.value].iloc[0]]=child 


    def pull_up(self, attribute):
      # print('in pull upp')
      if self.value==(attribute):
        return
      if self.value == None:
        self.expand(attribute)
        return 
      for value, child in self.children.items():
        child.pull_up(attribute)
      new_trees = []
      for branch_name, child in self.children.items():
        for child_branch_name, child_child in child.children.items():
          node = My_Node()
          node.value = self.value
          node.depth = self.depth + 1
          node.attslist = self.attslist.copy()
          node.attslist.remove(child.value)
          node.children[branch_name] = child_child
          node.parent = child
          node.X = child_child.X
          node.y = child_child.y
          child.parent = self.parent
          child.children[child_branch_name] = node
          child.depth = self.depth
          child.attslist = self.attslist.copy()
          child_child.parent = node
          # child.print()
        new_trees.append(child)
      new_root = My_Node.merge(new_trees)
      self.attslist = new_root.attslist
      self.value = new_root.value
      if self.value==None:
        print("OHHHHHHHHHHHHHH")
      self.label = None
      self.depth = new_root.depth
      self.children = new_root.children
      self.parent = new_root.parent

      return



    def merge(trees):
      # print('in merge')
      edge_values = []
      for i in range(len(trees)):
        if trees[i].children.keys() not in edge_values:
          edge_values += list(trees[i].children.keys())
      result = trees[0]    
      for tree in trees[1:]:
        result.X = pd.concat([result.X,tree.X])
        result.y = pd.concat([result.y,tree.y])
        for branch_name in edge_values:
          if branch_name in tree.children.keys():
            if branch_name in result.children.keys():
              result.children[branch_name].X = pd.concat([result.children[branch_name].X,tree.children[branch_name].X])
              result.children[branch_name].y = pd.concat([result.children[branch_name].y,tree.children[branch_name].y])
            else:
              result.children[branch_name] = tree.children[branch_name]
              result.children[branch_name].parent = result
            child_branch_name = list(tree.children[branch_name].children.keys())[0]
            result.children[branch_name].children[child_branch_name] = tree.children[branch_name].children[child_branch_name]
            result.children[branch_name].children[child_branch_name].parent = result.children[branch_name]
      My_Node.contraction(result)
 

      return result


    def contraction(result):
      # print('in contraction')
      for child in result.children.values():
        if (child.y.shape[0]==child.y[child.y['Label']=='yes'].shape[0]):
          child.label=1
          child.value=None
          child.children={}

        elif (child.y.shape[0]==child.y[child.y['Label']=='no'].shape[0]):
          child.label=0
          child.value=None
          child.children={}
                
    def iterate_on_nodes(root, parent_node=None):

      # print(root.X.shape)
      if root.value==None:
        if(root.label==0):
          return Node('-', parent_node)
        else:
          return Node('+', parent_node)
      root_node=Node(root.value, parent=parent_node)
      for branch_name in root.children.keys():
        branch_node=Node(branch_name, parent=root_node)
        My_Node.iterate_on_nodes(root.children[branch_name], branch_node)
      return root_node

    def print(self):
      root_node=My_Node.iterate_on_nodes(self)
      for pre, fill, node in RenderTree(root_node):
        print("%s%s" % (pre, node.name))


    def predict(self, x_test,y_test):
      labs=['no','yes']
      if self.value==None:
        ret=labs[self.label]
        return ret
      val=x_test[self.value]
      # print(val)
      # print(self.children.keys())
      if val in self.children.keys():
        # print(self.children[val])
        return self.children[val].predict(x_test,y_test)
      else:
        # print('in else')  
        y=self.y.copy()
        pos=y[y=='yes'].count().iloc[0]
        # print(pos)
        neg=y[y=='no'].count().iloc[0]
        if pos>=neg:
          # print(2)
          return 'yes'
        # print(3)  
        return 'no'  

    def cal_acc(self,x_test,y_test):
       pred_list=[]
       for i in range(x_test.shape[0]):
         pred_list.append(self.predict(x_test.iloc[i],y_test.iloc[i]))
       pred_series=pd.Series(pred_list)
      #  print('equavality:')
      #  print(pred_series)
      #  print(y_test)
      #  print(pred_series[pred_series==y_test])
       c=pred_series[pred_series==y_test].size
      #  print(c)
      #  print(y_test.shape[0])
       acc=c/y_test.shape[0]
       return acc  


    def reducing_error_prone(self,depth):
      self.label=1
      if (self.value==None or self.depth==depth-1):
        self.value=None
        self.children={}
        y=self.y.copy()
        pos=y[y=='yes'].count().iloc[0]
        # print(pos)
        neg=y[y=='no'].count().iloc[0]
        if pos>=neg:
          # print(2)
          self.label=1
        # print(3)  
        self.label=0 
        
      for child in self.children.values():
        return child.reducing_error_prone(depth)   

    def print_shape(root):
      # print(self.value)
      if root.value==None:
        return
      # print(root.X.shape)
      for child in root.children.values():
        return child.print_shape()  

In [ ]:
tree.print()


Medu
├── 1
│   └── Fedu
│       ├── 2
│       │   └── +
│       └── 1
│           └── reason
│               ├── course
│               │   └── -
│               └── home
│                   └── Fjob
│                       └── other
│                           └── Mjob
│                               ├── at_home
│                               │   └── -
│                               └── other
│                                   └── +
├── 2
│   └── -
├── 3
│   └── Fedu
│       ├── 2
│       │   └── reason
│       │       ├── course
│       │       │   └── -
│       │       └── reputation
│       │           └── +
│       ├── 4
│       │   └── reason
│       │       └── course
│       │           └── Fjob
│       │               └── other
│       │                   └── Pstatus
│       │                       ├── A
│       │                       │   └── +
│       │                       └── T
│       │                           └── -
│       └── 3
│           └── -
├── 4
│   └── Fedu

In [ ]:
pd.concat([X_train,y_train],axis=1).head(40)

,school,sex,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,Label
0,GP,F,U,LE3,T,4,3,health,other,reputation,father,yes
1,GP,M,U,GT3,T,3,2,at_home,other,reputation,mother,yes
2,GP,F,U,GT3,T,4,4,teacher,services,course,mother,no
3,GP,F,U,GT3,T,3,3,other,other,reputation,mother,no
4,GP,F,U,LE3,T,4,2,teacher,services,reputation,mother,yes
5,GP,F,R,GT3,T,2,4,services,health,course,mother,no
6,GP,F,U,GT3,T,1,1,at_home,other,home,mother,no
7,GP,M,R,GT3,T,4,3,teacher,services,course,mother,yes
8,MS,F,R,GT3,T,4,4,teacher,at_home,reputation,mother,no
9,GP,F,U,LE3,T,1,1,other,other,home,mother,yes


In [ ]:
y=(y_test)
x=(X_test)

print(tree.cal_acc(x,y))
# tree.predict(x,y)

0.425531914893617


##Reduce Error Proning- 4 different depths accuracies

In [ ]:
def error_prone(tree,X_train,y_train,x,y):
  tree.reducing_error_prone(depth=10)

  print('acc for train set after Reduce error proning with depth 10')
  print(tree.cal_acc(X_train, y_train))
  print('acc for test set after Reduce error proning with depth 10')
  print(tree.cal_acc(x,y))


  tree.reducing_error_prone(depth=6)
  print('acc for train set after Reduce error proning with depth 6')
  print(tree.cal_acc(X_train, y_train))
  print('acc after Reduce error proning with depth 6')
  print(tree.cal_acc(x,y))


  tree.reducing_error_prone(depth=5)
  print('acc for train set after Reduce error proning with depth 4')
  print(tree.cal_acc(X_train, y_train))
  print('acc after Reduce error proning with depth 5')
  print(tree.cal_acc(x,y))

  tree.reducing_error_prone(depth=4)
  print('acc for train set after Reduce error proning with depth 4')
  print(tree.cal_acc(X_train, y_train))
  print('acc after Reduce error proning with depth 4')
  print(tree.cal_acc(x,y))

  tree.reducing_error_prone(depth=2)
  print('acc for train set after Reduce error proning with depth 2')
  print(tree.cal_acc(X_train, y_train))
  print('acc after Reduce error proning with depth 2')
  print(tree.cal_acc(x,y))

  tree.reducing_error_prone(depth=1)
  print('acc for train set after Reduce error proning with depth 1')
  print(tree.cal_acc(X_train, y_train))
  print('acc after Reduce error proning with depth 1')
  print(tree.cal_acc(x,y))

##5 fold cross validation

In [ ]:
def run_5fold_CV(X_train,y_train,X_test,y_test):
  tree_fold=Tree()
  # dataset=X_test.join(y_test).drop_duplicates(subset=dataset.drop(['Label'], axis=1).columns).reset_index(drop=True)
  print(dataset.shape[0])
  # data = dataset.drop_duplicates(subset=dataset.drop(['Label'], axis=1).columns)
  index=[-1,90,195,293,330]
  # index=[-1,4,9,14,19]
  df=dataset.copy()
  acc_list=[]
  for i in  (range(len(index)-1)):
    print('in'+str(i+1)+'fold')
    df=dataset.copy()
    from_=index[i]+1
    to_=index[i+1]
    test=df.copy()[from_:to_]
    df=df.drop(df.index[from_:to_])
    y_train=df['Label'].reset_index(drop=True)
    X_train=df.drop(['Label'],axis=1).reset_index(drop=True)
    # print(X_train.shape[0])
    y_test=test['Label'].reset_index(drop=True)
    X_test=test.drop(['Label'],axis=1).reset_index(drop=True)
    tree_fold.train(X_train, y_train)
    acc=tree_fold.cal_acc(X_test,y_test)
    acc_list.append(acc)
    # print(df.shape)
  print('5-fold cross validation acc on test set is:')  
  print(sum(acc_list)/len(acc_list)  )

